In [78]:
from database import database

database= database("data.db")
database.connect()
resultats=database.read_json("all_matches_of_Cahier des charges_nouveau poste 2025_V0.6_définitive.pdf.json")
resultats






,title,question,sentence,score,pts
0,Analyse et synthèse,La fonction requiert la faculté d'analyser des...,Parfaite maîtrise de la langue française et ca...,0.519552,42.757
1,Analyse et synthèse,La fonction requiert la faculté d'analyser des...,Pensée analytique et conceptuelle,0.497364,42.757
2,Analyse et synthèse,La fonction requiert la faculté d'analyser des...,Excellente aptitude à l’expression orale,0.475437,42.757
3,Analyse et synthèse,La fonction requiert la faculté d'analyser des...,Compétences de communication avérées et aisanc...,0.466692,42.757
4,Analyse et synthèse,La fonction requiert la faculté d'analyser des...,Neuchâtel afin d’identifier les nouvelles comp...,0.443777,29.601
...,...,...,...,...,...
169,Rôle dans la gestion des ressources humaines,Elle a un rôle d'animation incluant le transfe...,En qualité de « responsable de la formation »,0.534832,11.232
170,Rôle dans la gestion des ressources humaines,Elle a un rôle d'animation incluant le transfe...,publiques actives dans le domaine de la formation,0.532336,11.232
171,Rôle dans la gestion des ressources humaines,Elle a un rôle de conduite de collaborateurs/t...,Soutenir activement le-la chef-fe du service d...,0.516199,24.336
172,Rôle dans la gestion des ressources humaines,Elle a un rôle de conduite de collaborateurs/t...,En qualité de « Chargé-e de projets RH »,0.508439,24.336


In [79]:
import spacy
nlp = spacy.load("fr_core_news_md")  # pas besoin d'exclude

def lemmatize(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc])

def to_lower(text):
    return text.lower()


In [80]:
resultats["sentence_lemmatized"] = resultats["sentence"].apply(to_lower)
resultats["sentence_lemmatized"] = resultats["sentence_lemmatized"].apply(lemmatize)
resultats['question_lemmatized'] = resultats['question'].apply(to_lower)
resultats['question_lemmatized'] = resultats['question_lemmatized'].apply(lemmatize)
resultats

,title,question,sentence,score,pts,sentence_lemmatized,question_lemmatized
0,Analyse et synthèse,La fonction requiert la faculté d'analyser des...,Parfaite maîtrise de la langue française et ca...,0.519552,42.757,parfait maîtrise de le langue français et capa...,le fonction requérir le faculté de analyser un...
1,Analyse et synthèse,La fonction requiert la faculté d'analyser des...,Pensée analytique et conceptuelle,0.497364,42.757,pensée analytique et conceptuel,le fonction requérir le faculté de analyser un...
2,Analyse et synthèse,La fonction requiert la faculté d'analyser des...,Excellente aptitude à l’expression orale,0.475437,42.757,excellent aptitude à l’ expression oral,le fonction requérir le faculté de analyser un...
3,Analyse et synthèse,La fonction requiert la faculté d'analyser des...,Compétences de communication avérées et aisanc...,0.466692,42.757,compétence de communication avérer et aisance ...,le fonction requérir le faculté de analyser un...
4,Analyse et synthèse,La fonction requiert la faculté d'analyser des...,Neuchâtel afin d’identifier les nouvelles comp...,0.443777,29.601,neuchâtel afin d’ identifier le nouveau compét...,le fonction requérir le faculté de analyser un...
...,...,...,...,...,...,...,...
169,Rôle dans la gestion des ressources humaines,Elle a un rôle d'animation incluant le transfe...,En qualité de « responsable de la formation »,0.534832,11.232,en qualité de « responsable de le formation »,lui avoir un rôle de animation inclure le tran...
170,Rôle dans la gestion des ressources humaines,Elle a un rôle d'animation incluant le transfe...,publiques actives dans le domaine de la formation,0.532336,11.232,public actif dans le domaine de le formation,lui avoir un rôle de animation inclure le tran...
171,Rôle dans la gestion des ressources humaines,Elle a un rôle de conduite de collaborateurs/t...,Soutenir activement le-la chef-fe du service d...,0.516199,24.336,soutenir activement le - le chef - fe de servi...,lui avoir un rôle de conduite de collaborateur...
172,Rôle dans la gestion des ressources humaines,Elle a un rôle de conduite de collaborateurs/t...,En qualité de « Chargé-e de projets RH »,0.508439,24.336,en qualité de « charger - e de projet rh »,lui avoir un rôle de conduite de collaborateur...


In [81]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [82]:
KEYWORDS = {
    "Formation de base":          ["AFP", "CFC", "Bachelor", "Master", "doctorat", "université", "HES", "ES"],
    "Formation complémentaire":   ["complémentaire","complétée","certification", ],
    "Durée":                      [ "ans", "année"],
    "Nature":                     ["professionnelle","expérience", "préalable", "même type"],
    "Autonomie de décision":      ["consignes", "directives", "autonome", "autonomie","approuve"],
    "Responsabilités budgétaires":["budget", "financier", "facturation", "paiements"],
    # "Responsabilités de planification à court terme":
    #                                 ["court terme", "plan", "optimisation", "procédure"],
    # "Responsabilités de planification à long terme":
    #                                 ["long terme", "prospective", "anticiper", "recherches", "études"],
    # "Impact externe des prestations":
    #                                 ["image", "représentatif", "conséquences", "tiers"],
    # "Impact interne des prestations":
    #                                 ["coûts", "bon fonctionnement", "irréversibles"],
    "Connaissances linguistiques":["français","française", "langue", "bilingue", "trilingue"],
    "Nature des communications internes":["communications", "échanges", "négociations", "décisions"],
    "Nature des communications externes":["communications", "informer", "explication", "négociations"],
    "Complexité de l'environnement":["difficultés", "adaptabilité", "flexibilité","complexe"],
    # "Evolution de l'environnement":["évolution", "processus", "rapide"],
    "Diversité des missions":     ["missions", "tâches", "diverses"],
    # "Diversité et quantité des postes à gérer":
    #                                 ["poste", "gère", "grand nombre", "activités"],
    "Rôle dans la gestion des ressources humaines":
                                    ["animation", "recrutement", "humaines", "ressources humaines"],
    "Innovation":                 ["innovation", "innovatrice"]
}


In [87]:
import re

def contains_keywords(text, text2, keywords, title):
    text = text.lower()
    if text2 is not None:
        text2 = text2.lower()
    if title in keywords:
        for keyword in keywords[title]:
            pattern = r'\b' + re.escape(keyword.lower()) + r's?\b'
            if re.search(pattern, text) and (re.search(pattern, text2) if text2 is not None else True):
                print(f"Found keyword '{keyword}' in title '{title}'")
                return True
    return False

In [88]:

def tfidf_similarity(text1, text2):
    vect = TfidfVectorizer()
    tfidf = vect.fit_transform([text1, text2])

    return cosine_similarity(tfidf[0], tfidf[1])[0][0]

resultats

,title,question,sentence,score,pts,sentence_lemmatized,question_lemmatized
0,Analyse et synthèse,La fonction requiert la faculté d'analyser des...,Parfaite maîtrise de la langue française et ca...,0.519552,42.757,parfait maîtrise de le langue français et capa...,le fonction requérir le faculté de analyser un...
1,Analyse et synthèse,La fonction requiert la faculté d'analyser des...,Pensée analytique et conceptuelle,0.497364,42.757,pensée analytique et conceptuel,le fonction requérir le faculté de analyser un...
2,Analyse et synthèse,La fonction requiert la faculté d'analyser des...,Excellente aptitude à l’expression orale,0.475437,42.757,excellent aptitude à l’ expression oral,le fonction requérir le faculté de analyser un...
3,Analyse et synthèse,La fonction requiert la faculté d'analyser des...,Compétences de communication avérées et aisanc...,0.466692,42.757,compétence de communication avérer et aisance ...,le fonction requérir le faculté de analyser un...
4,Analyse et synthèse,La fonction requiert la faculté d'analyser des...,Neuchâtel afin d’identifier les nouvelles comp...,0.443777,29.601,neuchâtel afin d’ identifier le nouveau compét...,le fonction requérir le faculté de analyser un...
...,...,...,...,...,...,...,...
169,Rôle dans la gestion des ressources humaines,Elle a un rôle d'animation incluant le transfe...,En qualité de « responsable de la formation »,0.534832,11.232,en qualité de « responsable de le formation »,lui avoir un rôle de animation inclure le tran...
170,Rôle dans la gestion des ressources humaines,Elle a un rôle d'animation incluant le transfe...,publiques actives dans le domaine de la formation,0.532336,11.232,public actif dans le domaine de le formation,lui avoir un rôle de animation inclure le tran...
171,Rôle dans la gestion des ressources humaines,Elle a un rôle de conduite de collaborateurs/t...,Soutenir activement le-la chef-fe du service d...,0.516199,24.336,soutenir activement le - le chef - fe de servi...,lui avoir un rôle de conduite de collaborateur...
172,Rôle dans la gestion des ressources humaines,Elle a un rôle de conduite de collaborateurs/t...,En qualité de « Chargé-e de projets RH »,0.508439,24.336,en qualité de « charger - e de projet rh »,lui avoir un rôle de conduite de collaborateur...


In [89]:
similarities = []
for idx, row in resultats.iterrows():
    sim = tfidf_similarity(row['sentence_lemmatized'], row['question_lemmatized'])
    if contains_keywords(row['sentence'], row['question'], KEYWORDS, row['title']):
        sim += 1
    elif contains_keywords(row['sentence'],None, KEYWORDS, row['title']):
        sim += 0.5
    similarities.append(sim)
resultats['similarity'] = similarities
resultats=resultats.sort_values(by='similarity', ascending=False)
resultats_needed = resultats[["title", "question", "sentence", "score", "pts", "similarity"]]
resultats_needed.sort_values(by='similarity', ascending=False, inplace=True)
print(resultats_needed[resultats_needed['title'] == "Formation complémentaire"].head(10))

Found keyword 'complexe' in title 'Complexité de l'environnement'
Found keyword 'française' in title 'Connaissances linguistiques'
Found keyword 'ans' in title 'Durée'
Found keyword 'complétée' in title 'Formation complémentaire'
Found keyword 'Bachelor' in title 'Formation de base'
Found keyword 'professionnelle' in title 'Nature'
Found keyword 'budget' in title 'Responsabilités budgétaires'
Found keyword 'humaines' in title 'Rôle dans la gestion des ressources humaines'
Found keyword 'humaines' in title 'Rôle dans la gestion des ressources humaines'
                       title  \
63  Formation complémentaire   
60  Formation complémentaire   
61  Formation complémentaire   
62  Formation complémentaire   
64  Formation complémentaire   

                                             question  \
63  Une formation complémentaire est nécessaire po...   
60  Deux formations complémentaires, ou plus, sont...   
61  Une formation complémentaire est nécessaire po...   
62  Une formation com

/var/folders/hw/687qrsn947v91l2yv331ngdc0000gn/T/ipykernel_73412/4042138206.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resultats_needed.sort_values(by='similarity', ascending=False, inplace=True)


In [90]:
resultats_fin = resultats_needed.sort_values('similarity', ascending=False).drop_duplicates(subset=['title'], keep='first')
resultats_fin

,title,question,sentence,score,pts,similarity
52,Durée,Une durée minimum de 1 à 3 ans d'expérience pr...,Minimum 3 à 5 ans de pratique professionnelle ...,0.746009,7.80000,1.607958
104,Nature,La fonction requiert de l'expérience professio...,Minimum 3 à 5 ans de pratique professionnelle ...,0.667060,11.70000,1.442260
65,Formation de base,La fonction requiert un Bachelor (Université o...,Formation supérieure de niveau Bachelor ou Mas...,0.650470,20.93000,1.342312
167,Rôle dans la gestion des ressources humaines,Elle est peu ou pas concernée par la gestion d...,Service des ressources humaines (SRH),0.541663,0.00000,1.249635
28,Complexité de l'environnement,La fonction fait fréquemment face à un degré i...,Sens des responsabilités et autonomie dans la ...,0.437550,50.05000,0.742610
129,Responsabilités budgétaires,La fonction participe à l'élaboration et assur...,Suivre l’évolution du budget de formation en c...,0.522343,33.69600,0.709697
34,Connaissances linguistiques,La fonction requiert des connaissances linguis...,Parfaite maîtrise de la langue française et ca...,0.759499,4.42000,0.626473
63,Formation complémentaire,Une formation complémentaire est nécessaire po...,idéalement complétée par une certification,0.498503,9.75000,0.567436
153,Responsabilités de planification et de réalisa...,La fonction est responsable de l'élaboration e...,participer à la définition et à la mise en œuv...,0.526287,23.86800,0.511010
13,Autonomie de décision,"La fonction approuve, pour l'/les institution/...",participer à la définition et à la mise en œuv...,0.429479,93.60000,0.501472


In [91]:
resultats_fin.to_json("results/after_comparaison_lexicale_of_Test6.docx.json", orient="records", force_ascii=False, indent=4)